# Distributions

In [1]:
from IPython import get_ipython
if get_ipython():
    get_ipython().run_line_magic("load_ext", "autoreload")
    get_ipython().run_line_magic("autoreload", "2")

import numpy as np
import pandas as pd
import torch
import math

import xarray as xr

import matplotlib.pyplot as plt
import seaborn as sns

import collections

import latenta as la
la.logger.setLevel("INFO")

In [2]:
cells = la.Dim(pd.Series(range(3), name = "cell").astype(str))
genes = la.Dim(pd.Series(range(4), name = "gene").astype(str))

## Basic distributions

In [3]:
dist = la.distributions.Normal(loc = 1.)

In [4]:
dist.reset()
dist.run()
print(dist.value, dist.likelihood)

tensor(1.3355) tensor(-0.9752)


In [5]:
dist.prior_mean

tensor(1.)

### Automatic broadcasting

In [6]:
dist = la.distributions.Normal(definition = la.Definition([cells]))

In [7]:
dist.reset()
dist.run()
print(dist.value)
print(dist.likelihood)

tensor([-0.2425, -0.2161, -2.1495])
tensor([-0.9483, -0.9423, -3.2292])


In [8]:
dist = la.distributions.Normal(loc = la.Fixed(10., definition = la.Definition([cells])))

In [9]:
dist.reset()
dist.run()
print(dist.value)
print(dist.likelihood)

tensor([ 9.1606, 11.1253, 10.0205])
tensor([-1.2712, -1.5521, -0.9191])


In [10]:
dist = la.distributions.Normal(loc = la.Fixed(10., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])))
dist = la.distributions.Laplace(loc = la.Fixed(10., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])))
dist = la.distributions.Uniform(high = la.Fixed(10., definition = la.Definition([cells])), low = la.Fixed(2., definition = la.Definition([genes])))

In [11]:
dist.reset()
dist.run()
print(dist.value)
print(dist.likelihood)
print(dist.icdf(0.9))
print(dist.spread(3))

tensor([[7.2276, 4.8883, 9.1352],
        [8.6058, 4.1857, 6.7979],
        [3.0507, 5.6252, 9.8647],
        [8.6184, 7.6132, 3.5194]])
tensor([[-2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794],
        [-2.0794, -2.0794, -2.0794]])
tensor([[9.2000, 9.2000, 9.2000],
        [9.2000, 9.2000, 9.2000],
        [9.2000, 9.2000, 9.2000],
        [9.2000, 9.2000, 9.2000]])
tensor([[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.],
         [ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 6.,  6.,  6.],
         [ 6.,  6.,  6.],
         [ 6.,  6.,  6.],
         [ 6.,  6.,  6.]],

        [[10., 10., 10.],
         [10., 10., 10.],
         [10., 10., 10.],
         [10., 10., 10.]]])


In [12]:
dist.redefine(la.Definition([cells, genes]))
dist.redefine(la.Definition([genes, cells]))

??: high, low ↦ Uniform [gene (•), cell (•)]

### Log probability with new dimensions

In [13]:
dist_1 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])))
dist_2 = la.distributions.Normal(loc = la.Fixed(np.linspace(-1, 1., len(cells)), definition = la.Definition([cells])), definition = dist_1.clean)
dist_3 = la.distributions.Normal(loc = la.Fixed(np.linspace(-1, 1., len(genes)), definition = la.Definition([genes])), definition = dist_1.clean)

In [14]:
dist_1.reset()
dist_1.run()

dist_2.reset()
dist_2.run()

dist_3.reset()
dist_3.run()

In [15]:
dist_2.likelihood

tensor([[-0.9605],
        [-1.1367],
        [-1.0209]])

In [16]:
(dist_2.likelihood == dist_2.log_prob(dist_2.value)).all()

tensor(True)

In [17]:
dist_2.log_prob(dist_1.value)

tensor([[-11.8024,  -2.6410,  -1.1641,  -1.0243],
        [ -0.9787,  -1.2652,  -7.4973,  -1.6329],
        [ -8.2210,  -9.1332,  -1.2835,  -0.9365]])

In [18]:
dist_3.likelihood

tensor([[-1.2532, -1.0677, -1.4362, -2.4648]])

In [19]:
(dist_3.likelihood == dist_3.log_prob(dist_3.value)).all()

tensor(True)

In [20]:
dist_3.log_prob(dist_1.value)

tensor([[-11.8024,  -1.6260,  -2.9866,  -3.9422],
        [ -1.8245,  -1.5982,  -8.7619,  -0.9379],
        [ -2.5779,  -4.6178,  -0.9365,  -0.9365]])

### Dependent dimensions

We can mark some dimensions as dependent dimensions:

In [21]:
dist_1 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), dependent_dims = {cells})
dist_2 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])))

In [22]:
dist_1.reset()
dist_1.run()

dist_2.reset()
dist_2.run()

In [23]:
dist_1.likelihood

tensor([-3.1096])

In [24]:
dist_2.likelihood

tensor([-1.1387, -2.3708, -1.2797])

In contrast to pytorch and tensorflow, we do not assume that event dimensions are at the end of the tensor. They can be anywhere

In [25]:
dist_3 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {cells})
dist_4 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {genes})

In [26]:
dist_3.reset()
dist_3.run()

dist_4.reset()
dist_4.run()

In [27]:
dist_3.likelihood

tensor([[-4.8977, -5.4575, -7.7183, -5.9106]])

In [28]:
dist_4.likelihood

tensor([[-9.8481],
        [-7.3007],
        [-7.9980]])

You can always know which dimensions are dependent using `dist.event_dims`

In [29]:
dist.event_dims

set()

## Event-based distributions

Some distributions have default dependent dimensions

In [30]:
dist = la.distributions.Dirichlet(concentration = la.Fixed(1., definition = la.Definition([cells])))

In [31]:
dist.event_dims

{Dim cell}

In [32]:
dist.reset()
dist.run()
print(dist.value)
print(dist.likelihood)

tensor([0.1302, 0.7238, 0.1460])
tensor([0.6931])


In [33]:
value = torch.ones(dist.value_definition.shape)
value = value / value.sum(0, keepdim = True)
assert torch.isclose(torch.exp(dist.log_prob(value)), torch.tensor(2.))

We can of course add multiple dimensions to such a distribution. By default (as per tensorflow/torch standards), the dependent dimension will be the last one:

In [34]:
dist = la.distributions.Dirichlet(concentration = la.Fixed(10., definition = la.Definition([cells, genes])))

In [35]:
dist.component_dim

Dim gene

In [36]:
dist.reset()
dist.run()
print(dist.value)
print(dist.value.sum(1))
print(dist.likelihood)

tensor([[0.3283, 0.2667, 0.2364, 0.1687],
        [0.1905, 0.3498, 0.1696, 0.2901],
        [0.1537, 0.4054, 0.2307, 0.2102]])
tensor([1., 1., 1.])
tensor([[4.5045],
        [3.9425],
        [3.2066]])


However, we can manually choose the dependent dimension as well:

In [37]:
dist = la.distributions.Dirichlet(concentration = la.Fixed(10., definition = la.Definition([cells, genes])), component_dim = cells)

In [38]:
dist.reset()
dist.run()
print(dist.value)
print(dist.value.sum(0))
print(dist.likelihood)

tensor([[0.3257, 0.3473, 0.3579, 0.2238],
        [0.4070, 0.2486, 0.3593, 0.3791],
        [0.2673, 0.4041, 0.2828, 0.3971]])
tensor([1., 1., 1., 1.])
tensor([[2.7905, 2.6518, 3.0246, 2.3373]])


We can also still add other dependent dimensions

In [39]:
dist = la.distributions.Dirichlet(concentration = la.Fixed(10., definition = la.Definition([cells, genes])), component_dim = cells, dependent_dims = {genes})

In [40]:
dist.reset()
dist.run()
print(dist.value)
print(dist.value.sum(0))
print(dist.likelihood)

tensor([[0.3792, 0.4597, 0.3200, 0.2853],
        [0.3303, 0.2497, 0.2885, 0.3752],
        [0.2904, 0.2905, 0.3915, 0.3396]])
tensor([1., 1., 1., 1.])
tensor([[11.2628]])


## Transformed distributions

We can directly transform the output of a distribution, using invertible transforms

In [41]:
dist_1 = la.distributions.Normal(scale = 1., transforms = [la.transforms.Affine(a = 2.)])
dist_2 = la.distributions.Normal(scale = 2.)

In [42]:
dist_1.reset()
torch.manual_seed(0)
dist_1.run()

dist_2.reset()
torch.manual_seed(0)
dist_2.run()

In [43]:
dist_1.likelihood

tensor(-2.7994)

In [44]:
dist_1.log_prob(dist_1.value)

tensor(-2.7994)

In [45]:
dist_2.log_prob(dist_1.value)

tensor(-2.7994)

In [46]:
assert dist_1.likelihood == dist_1.log_prob(dist_1.value)
assert dist_1.likelihood == dist_2.log_prob(dist_1.value)

----------

In [47]:
dist_1 = la.distributions.LogNormal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {cells})
dist_2 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {cells}, transforms = [la.transforms.Exp()])

In [48]:
dist_1.transforms

[Exp]

In [49]:
dist_1.reset()
dist_1.run()

dist_2.reset()
dist_2.run()

In [50]:
dist_1.likelihood

tensor([[ -7.0021, -10.0257, -15.2661,  -6.5818]])

In [51]:
dist_2.likelihood

tensor([[-36.1292, -13.0858, -12.3198,  -8.9711]])

-----

In [52]:
dist_1 = la.distributions.LogitNormal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {cells})
dist_2 = la.distributions.Normal(loc = la.Fixed(0., definition = la.Definition([cells])), scale = la.Fixed(2., definition = la.Definition([genes])), dependent_dims = {cells}, transforms = [la.transforms.Logistic()])

In [53]:
dist_1.transforms

[Logistic]

In [54]:
dist_1.reset()
dist_1.run()

dist_2.reset()
dist_2.run()

In [55]:
dist_1.likelihood

tensor([[ 0.1718,  0.5671,  0.1295, -0.2759]])

In [56]:
dist_2.log_prob(dist_1.value)

tensor([[ 0.1718,  0.5671,  0.1295, -0.2759]])

------------

In [57]:
axes = la.Dim(["x", "y"], id = "axis")

In [58]:
dist_1 = la.distributions.Normal(
    loc = la.Fixed(1., definition = la.Definition([cells, axes])),
    scale = la.Fixed(0.1, definition = la.Definition([genes])),
    transforms = [la.transforms.Circular()],
    dependent_dims = {genes}
)

In [59]:
dist_1

??: loc, scale ↦ Normal [cell (•), gene (•)]

In [60]:
dist_1.reset()
dist_1.run()

In [61]:
dist_1.value

tensor([[0.8403, 0.8346, 0.7995, 0.7897],
        [0.9381, 0.9291, 0.8108, 0.7928],
        [0.7733, 0.8137, 0.7260, 0.8454]])

In [62]:
dist_1.likelihood

tensor([[10.4321],
        [ 6.6747],
        [11.0030]])

## Reparameterization

If a distribution is not reparameterizable, the validation will check whether any upstream components are optimizable.

## Redefine

Distributions are used to sample or to calculate log probabilities. If a distribution is a component, it is often redefined to match the desired definition:

In [64]:
p = la.distributions.Normal()
observation = la.Observation(
    value = pd.Series(0., index = genes.index),
    p = p
)
observation.p

p: loc, scale ↦ Normal [gene (4)]

In [66]:
original_p = la.distributions.Normal()
original_q = la.distributions.Normal(1.)

In [67]:
latent = la.Latent(
    q = original_q,
    p = original_p,
    definition = la.Definition([genes])
)
print(latent.p)
print(latent.q)

p: loc, scale ↦ Normal [gene (4)]
q: loc, scale ↦ Normal [gene (4)]


Be aware that after redefinition, the distribution is not the one you provided:

In [73]:
assert original_p != latent.p

However, the upstream components will be still be the same:

In [74]:
assert original_p.loc == latent.p.loc